In [32]:
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/
#Used tutorial given above

In [33]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [34]:
filename="audience"
testname="audienceTest"
output_dir = "/content/drive/My Drive/audience/"
path="/content/drive/My Drive/TrainTest/"
Trainp=path+filename+".csv"
Testp=path+testname+".csv"

In [35]:
#From hugging face library for pre trained transformers package.
!pip install transformers
!pip install  pytorch_pretrained_bert

In [36]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [37]:
#See if GPU allocated
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [38]:
#df,dtest=train_test_split(data,test_size=0.20,random_state=2018)
df=pd.read_csv(Trainp,delimiter=',')
dtest=pd.read_csv(Testp,delimiter=',')
print ("train: " ,df.shape )
print ("test: ",dtest.shape)

train:  (3050, 5)
test:  (763, 5)


In [39]:
df.sample(10)

,Unnamed: 0,Unnamed: 0.1,segment_id,segment_txt,label
1517,3268,3268,26,If you wish to limit how third parties use you...,0
2369,2351,2351,36,Correcting/updating personal information If ...,0
1961,2240,2240,52,9. WHERE YOUR INFORMATION WILL BE USED NBCUn...,1
343,421,421,16,Please send comments or questions to DCCCD's D...,0
2663,671,671,2,Collection of Information By using Valve's o...,0
1173,1770,1770,12,"We use web beacons (also known as ""action ta...",0
2676,238,238,30,You may also correspond by standard post at: ...,0
1027,2959,2959,32,"We predominately use ""session"" cookies, which ...",0
2083,2704,2704,23,If there are any issues related to your user a...,0
1444,79,79,23,"Some of the content, advertising, and function...",0


In [40]:
#Allocate GPU to torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [41]:
sentences=df.segment_txt.values
#SEP for  two sentence tasks
#CLS for classification. At the nd of 12 layers CLS 768 embedding vector 
#segment=["[CLS]" + sentence + "[SEP]" for sentence in segment ]
labels=df.label.values

In [43]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
 # encode will: Tokenize , Prepend the [CLS], [SEP],map to ids
# For every sentence...
for sent in sentences:
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True,)
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)
# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  For more information about some of our most popular services, you can visit the Google Product Privacy Guide.  
Token IDs: [101, 2005, 2062, 2592, 2055, 2070, 1997, 2256, 2087, 2759, 2578, 1010, 2017, 2064, 3942, 1996, 8224, 4031, 9394, 5009, 1012, 102]


In [44]:
segmentlength=[len(sent) for sent in input_ids]
print (max(segmentlength))

442


In [45]:
MAX_LEN=300

In [47]:
from keras.preprocessing.sequence import pad_sequences
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


In [48]:
#Mask of 1s for input and 0 for padding
attention_masks = []
#For each sentence
for seq in input_ids:
  #if a token is 0 then set mask 0, 1 otjerwise  
  seq_mask = [int(i>0) for i in seq]
  attention_masks.append(seq_mask)
print (attention_masks[5])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [49]:
#Splitting into training and validaton data
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1,shuffle=False)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,random_state=2018, test_size=0.1,shuffle=False)
#Converting to torch sensors
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [50]:
#Recommended batch size 16 or 32
batch_size=16
#MAking an Iterator for the training and validation data
#Will help with memory because unlike for loop entire dataset doesnt need to be in memory
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [51]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
#Bert with additonal classification layer
#CLS with 768,2
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2,output_attentions=False,output_hidden_states=False)
#Telling model to run on cuda
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [52]:
from transformers import AdamW
optimizer=AdamW(model.parameters(),lr=2e-5)

In [53]:
#Calculating accuracy of prediction 
def flat_accuracy(preds, labels):
  pred_flat=np.argmax(preds,axis=1).flatten()
  labels_flat=labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [54]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [55]:
from transformers import get_linear_schedule_with_warmup
t = [] 
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [56]:
import random
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. 
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
      
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
      
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")



======== Epoch 1 / 4 ========
Training...
  Batch    40  of    172.    Elapsed: 0:00:20.
  Batch    80  of    172.    Elapsed: 0:00:39.
  Batch   120  of    172.    Elapsed: 0:00:59.
  Batch   160  of    172.    Elapsed: 0:01:19.

  Average training loss: 0.18
  Training epcoh took: 0:01:24

Running Validation...
  Accuracy: 0.97
  Validation took: 0:00:03

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    172.    Elapsed: 0:00:20.
  Batch    80  of    172.    Elapsed: 0:00:39.
  Batch   120  of    172.    Elapsed: 0:00:59.
  Batch   160  of    172.    Elapsed: 0:01:18.

  Average training loss: 0.07
  Training epcoh took: 0:01:24

Running Validation...
  Accuracy: 0.97
  Validation took: 0:00:03

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    172.    Elapsed: 0:00:20.
  Batch    80  of    172.    Elapsed: 0:00:39.
  Batch   120  of    172.    Elapsed: 0:00:59.
  Batch   160  of    172.    Elapsed: 0:01:18.

  Average training loss: 0.04
  Training epcoh t

In [57]:
import os
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME
#output_dir = "/content/drive/My Drive/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir,WEIGHTS_NAME)
output_config_file = os.path.join(output_dir,CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

('/content/drive/My Drive/audience/vocab.txt',)

In [60]:
sentences = dtest.segment_txt.values
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
#sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = dtest.label.values
#tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)



MAX_LEN = 300
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
#input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",value=0, truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [int(i>0) for i in seq]
  attention_masks.append(seq_mask) 
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
batch_size = 16
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [61]:
#model.save('BERT.h5')
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [62]:
#print (len(predictions[0])) #24*32
flat_predictions = [item for sublist in predictions for item in sublist] #763*2
#print ((flat_predictions))
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
#print ((flat_predictions))
flat_true_labels = [item for sublist in true_labels for item in sublist]
#print (len(flat_true_labels))
from sklearn.metrics import classification_report
print (classification_report(flat_true_labels,flat_predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       687
           1       0.94      0.89      0.92        76

    accuracy                           0.98       763
   macro avg       0.97      0.94      0.96       763
weighted avg       0.98      0.98      0.98       763

